In [1]:
#general libraries
import pandas as pd
import numpy as np
import copy
import wandb
#project code
from FNN import FNN
from gradient_descent import *
from ActivFunctions import  *
from LossFunctions import *
from SuppFunctions import *
from TestingFunctions import *

In [ ]:
api_key = ""#specify your api_key, but never share it
entity = "DL_project_Group_70"
project = "test"

In [ ]:
#setting general network hyperparameters'
epochs = 20
batch_size = 128
loss_derivative = SoftmaxCrossEntropyDerivative
size_test_set = 0.1
#sgd specific
learning_rate_sgd = 0.0001
grad_clip_sgd = 10
#sgd_momentum specific
learning_rate_sgd_momentum = 0.0001
grad_clip_sgd_momentum = 10
momentum_sgd_momentum = 0.95
#rmsprop specific
learning_rate_rmsprop = 0.0001
grad_clip_rmsprop = 0
#nag specific
learning_rate_nag = 0.0001
grad_clip_nag = 10
momentum_nag = 0.6
#adam specific
learning_rate_adam = 0.0001
grad_clip_adam = 0

In [ ]:
#getting the data for multi class classification (no need to have data physically downloaded)
from sklearn.datasets import fetch_openml
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [ ]:
#preparing dataset
dataInput = X.T
dataTarget = y[np.newaxis,:]
dataTarget_one_hot = one_hot_encode(dataTarget,10)

In [ ]:
#getting data split
n_samples = dataTarget.size
indices = np.arange(0,n_samples)
indices_test = np.random.permutation(round(n_samples*0.2))
indices_train = list(set(indices) - set(indices_test))

In [ ]:
#getting train set
dataInput_train = dataInput[:,indices_train]
dataTarget_train = dataTarget_one_hot[:,indices_train]
#getting test set
dataInput_test = dataInput[:,indices_test]
dataTarget_test = dataTarget_one_hot[:,indices_test]
#normalization
dataInput_train = dataInput_train/255.0
dataInput_test =  dataInput_test/255.0

In [ ]:
#declaring network
netClassMulti = FNN([784,1024,512, 256, 128, 10],
          [relu,softmax],
          method_ini="HeNor"
)

In [ ]:
net_classMulti_adam = train_minibatch_adam(copy.deepcopy(netClassMulti), dataInput_train, dataTarget_train, epochs, learning_rate_adam, batch_size, loss_derivative, grad_clip = grad_clip_adam)

In [ ]:
wandb.login(api_key)

True

In [ ]:
sweep_configuration = {
    "name": "sweepdemo",
    "method": "bayes",
    "metric": {"goal": "minimize", "name": "validation_loss"},
    "parameters": {
        "dataset": {"values": ["MNIST"]},#"MNIST","CIFAR"
        "epochs": {"values": [5, 10, 15]},
        "num_hidden_layers": {"values": [3, 4, 5]},
        "num_hidden_units": {"values": [16, 32, 64, 128, 256]},
        "optimizer": {"values": ["sgd","sgd_momentum","rmsprop","nag","adam"]},
        "learning_rate": {"min": 0.0001, "max": 0.01},
        "grad_clip": {"min": 0.5, "max": 5},
        "momentum": {"min": 0.8, "max": 0.95},#sgd_momentum,nag
        "beta": {"min": 0.9, "max": 0.99},#rmsprop
        "beta1": {"min": 0.8, "max": 0.95},#adam
        "beta2": {"min": 0.96, "max": 0.99},#adam
        "batch_size": {"values": [16, 32, 64, 128, 256]},
        "l_coeff": {"values": {"min": 1e-5, "max": 1e-3}},
        "l_method": {"values": ["none","l1", "l2"]},
        "weights_init": {"values": ["Zero", "RandomUni", "RandomNor", "XavUni", "XavNor", "HeUni", "HeNor"]},
        "activation_hidden": {"values": ["identity", "sigmoid", "tanh", "relu", "leaky_relu", "softmax"]},
        "activation_output": {"values": ["identity", "sigmoid", "tanh", "relu", "leaky_relu", "softmax"]},
        "leaky_relu_slope": {"min": 0.02, "max": 0.02},#leaky_relu
        "loss_function": {"values": ["MeanSquaredError", "CrossEntropy", "SoftmaxCrossEntropy"]},
    },
}